# relax計算のtrajectoryをディレクトリに分割する

In [1]:
from glob import glob
from mlptools.io.read import read_from_format
from ase.io.espresso import read_espresso_out, read_espresso_in
import os
import pickle

In [2]:
path2relax = "/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/Si/diamond__mp-149/relax/export"
relax_dirs = glob(os.path.join(path2relax, "relax_*"))
print(f"Number of relax directories: {len(relax_dirs)}")

Number of relax directories: 71


In [4]:
relax_dirs[:5]

['/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/Si/diamond__mp-149/relax/export/relax_16e46f04-d35d-45d1-8ef0-fbdbc0bd6f78',
 '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/Si/diamond__mp-149/relax/export/relax_16c8d448-890f-4d23-9203-96edb5fc95d7',
 '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/Si/diamond__mp-149/relax/export/relax_1bfa2d80-5183-4362-84cd-602aa3bfc67e',
 '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/Si/diamond__mp-149/relax/export/relax_18179055-da70-47cb-b30a-b400d6b8ac92',
 '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/Si/diamond__mp-149/relax/export/relax_0430b474-a1b1-4d82-9b32-f1b3d93d4937']

In [5]:
# 各relaxディレクトリに対して、trajectoryごとにディレクトリを作成しその中に、Atoms(ase_atoms.pkl)とMLPAtoms(atoms.pkl)を保存する
for relax_dir in relax_dirs:
    all_ase_atoms = []
    print(relax_dir)

    with open(os.path.join(relax_dir, "relax.out")) as f:
        lines = [s.strip() for s in f.readlines()]
    is_valid = True
    for l in lines:
        if "convergence NOT achieved" in l:
            is_valid = False
            break
    if not is_valid:
        continue
    
    with open(os.path.join(relax_dir, "relax.out")) as f:
        atom_gen = read_espresso_out(f, index=slice(None))
        while True:
            try:
                ase_atoms = next(atom_gen)
                all_ase_atoms.append(ase_atoms)
            except StopIteration:
                break
    
    print(f"Number of trajectories: {len(all_ase_atoms)}")
    if len(all_ase_atoms) == 0:
        continue

    # 全atomsを保存
    for i, ase_atoms in enumerate(all_ase_atoms):
        # create directory
        path2atoms = os.path.join(relax_dir, "atoms", f"atoms_{i}")
        os.makedirs(path2atoms, exist_ok=True)

        # create mlpatoms
        mlpatoms = read_from_format(
            ase_atoms=ase_atoms,
            format="ase",
        )

        # save atoms
        pickle.dump(ase_atoms, open(os.path.join(path2atoms, "ase_atoms.pkl"), "wb"))
        pickle.dump(mlpatoms, open(os.path.join(path2atoms, "atoms.pkl"), "wb"))

/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/Si/diamond__mp-149/relax/export/relax_16e46f04-d35d-45d1-8ef0-fbdbc0bd6f78


Number of trajectories: 49
/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/Si/diamond__mp-149/relax/export/relax_16c8d448-890f-4d23-9203-96edb5fc95d7
Number of trajectories: 31
/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/Si/diamond__mp-149/relax/export/relax_1bfa2d80-5183-4362-84cd-602aa3bfc67e
Number of trajectories: 14
/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/Si/diamond__mp-149/relax/export/relax_18179055-da70-47cb-b30a-b400d6b8ac92
Number of trajectories: 50
/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/Si/diamond__mp-149/relax/export/relax_0430b474-a1b1-4d82-9b32-f1b3d93d4937
Number of trajectories: 50
/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/Si/diamond__mp-149/relax/export/relax_09ffb340-8692-4281-a7c7-14efc89f63aa
Number of trajectories: 26
/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/Si/diamond__mp-149/relax/export/relax_3514cc54-3b9d-42e1-bd5d-a09aea02bdb8
Number of trajectories: 19
/Users/y1u0d2/Google Drive/マイドライブ/H

In [5]:
# for ase_atoms in all_ase_atoms:
#     mlpatoms = read_from_format(
#         ase_atoms=ase_atoms,
#         format="ase",
#     )
#     break